In [74]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import joblib

In [60]:
df1 = pd.read_csv('2014-15.csv')
df2 = pd.read_csv('2015-16.csv')
df3 = pd.read_csv('2016-17.csv')
df4 = pd.read_csv('2017-18.csv')
df5 = pd.read_csv('2018-19.csv')
df6 = pd.read_csv('2019-20.csv')
df7 = pd.read_csv('2021.csv')
df8 = pd.read_csv('2022.csv')
#loading our fifa dataset
df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8], axis=0 )
# sort the dataframe by hometeams 
df.sort_values(by=['HomeTeam'], ascending=True)
df.to_csv('merged.csv', index=False)
#preprocessing data 
selected_features = ['HomeTeam', 'AwayTeam','FTHG','FTAG','FTR','HTR','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR']
df = df[selected_features].dropna()
print(df)


           HomeTeam        AwayTeam  FTHG  FTAG FTR HTR  HS  AS  HST  AST  HF  \
0           Arsenal  Crystal Palace     2     1   H   D  14   4    6    2  13   
1         Leicester         Everton     2     2   D   A  11  13    3    3  16   
2        Man United         Swansea     1     2   A   A  14   5    5    4  14   
3               QPR            Hull     0     1   A   D  19  11    6    4  10   
4             Stoke     Aston Villa     0     1   A   D  12   7    2    2  14   
..              ...             ...   ...   ...  ..  ..  ..  ..  ...  ...  ..   
375  Crystal Palace      Man United     1     0   H   H   6  10    3    4  12   
376       Leicester     Southampton     4     1   H   D  12   7    6    2  10   
377       Liverpool          Wolves     3     1   H   D  29   7    8    5   6   
378        Man City     Aston Villa     3     2   H   A  24   4    5    2   5   
379         Norwich       Tottenham     0     5   A   A   9  19    0   13  13   

     AF  HC  AC  HY  AY  HR

In [56]:
#normalizing our data
scaler = StandardScaler()
normalized_data = pd.DataFrame(scaler.fit_transform(df[['FTHG','FTAG','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR']]))

# assign weights to each parameter (full-timegoals, shots, shots on target, fouls,corners, yellow cards, red cards )
weights = [0.25, 0.15, 0.2, 0.1, 0.1, 0.1, 0.1]
# calculate team ratings for each match
home_team_rating = normalized_data.iloc[:, [0, 2 , 4, 6, 8, 10, 12]].mul(weights).sum(axis=1)
away_team_rating = normalized_data.iloc[:, [1, 3 , 5 ,7, 9, 11, 13]].mul(weights).sum(axis=1)



In [73]:
# combine home and away team ratings into one dataframe
X = pd.concat([home_team_rating, away_team_rating], axis=1)

# convert FTR column into binary labels
df['FTR_binary'] = df['FTR'].apply(lambda x: 1 if x == 'H' else 0)

# split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, df['FTR_binary'], test_size=0.2, random_state=42)

# train logistic regression model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# evaluate model performance
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1_score,_= precision_recall_fscore_support(y_test, y_pred, average='binary')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)

Accuracy: 0.7648148148148148
Precision: 0.728110599078341
Recall: 0.6991150442477876
F1-score: 0.7133182844243793


In [75]:
#saving the model
joblib.dump(logreg, 'logreg_model.h5')

['logreg_model.h5']